In [64]:
import os
import openai
import torch
import google.generativeai as genai
import gc
from accelerate import dispatch_model
from tenacity import retry, stop_after_attempt, wait_exponential
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.cuda.amp import autocast

In [65]:
# Replace with your actual OpenAI API key
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
openai.api_key = os.getenv("OPENAI_API_KEY")

os.environ["GEMINI_API_KEY"] = "GEMINI_API_KEY"
genai.api_key=os.getenv("GEMINI_API_KEY")

In [66]:
# ========== Qwen ==========
qwen_model_path = r"cache/model/Qwen_Qwen2.5-72B-Instruct/models--Qwen--Qwen2.5-72B-Instruct/snapshots/495f39366efef23836d0cfae4fbe635880d2be31"
qwen_tokenizer_path = r"cache/tokenizer/Qwen_Qwen2.5-72B-Instruct/models--Qwen--Qwen2.5-72B-Instruct/snapshots/495f39366efef23836d0cfae4fbe635880d2be31"

# ========== Llama ==========
llama_model_path = r"cache/model/meta-llama_Llama-3.3-70B-Instruct/models--meta-llama--Llama-3.3-70B-Instruct/snapshots/6f6073b423013f6a7d4d9f39144961bfbfbc386b"
llama_tokenizer_path = r"cache/tokenizer/meta-llama_Llama-3.3-70B-Instruct/models--meta-llama--Llama-3.3-70B-Instruct/snapshots/6f6073b423013f6a7d4d9f39144961bfbfbc386b"


In [67]:
def clear_memory():
    """Giải phóng bộ nhớ GPU và CPU"""
    import gc
    import torch
    
    # Giải phóng bộ nhớ GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    
    # Giải phóng bộ nhớ CPU
    gc.collect()
    
    print("🧹 Đã giải phóng bộ nhớ cache")

: 

In [ ]:
 #Hàm load mô hình và tokenizer
def load_model_and_tokenizer(model_path, tokenizer_path, model_name):
    print(f"\n🔄 Đang load mô hình {model_name}...")
    try:
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(
            tokenizer_path,
            trust_remote_code=True,
            use_fast=False
        )
        
        # Load model
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map="balanced",  # Tự động phân phối trên các GPU
            torch_dtype=torch.bfloat16,  # Sử dụng half precision để tiết kiệm bộ nhớ
            trust_remote_code=True
        )
        
        print(f"✅ Đã load thành công mô hình {model_name}")
        return model, tokenizer
    except Exception as e:
        print(f"❌ Lỗi khi load mô hình {model_name}: {e}")
        return None, None

# Load Qwen
qwen_model, qwen_tokenizer = load_model_and_tokenizer(qwen_model_path, qwen_tokenizer_path, "Qwen")

# Load Llama
llama_model, llama_tokenizer = load_model_and_tokenizer(llama_model_path, llama_tokenizer_path, "Llama")

# Kiểm tra xem các mô hình đã được load thành công chưa
if qwen_model is None or qwen_tokenizer is None:
    print("⚠️ Không thể load mô hình Qwen")
if llama_model is None or llama_tokenizer is None:
    print("⚠️ Không thể load mô hình Llama")

INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
INFO:accelerate.utils.modeling:Based on the current allocation process, no modules could be assigned to the following devices due to insufficient memory:
  - 0: 4982833152 bytes required
  - 1: 2491416576 bytes required
  - 2: 2491416576 bytes required
These minimum requirements are specific to this allocation attempt and may vary. Consider increasing the available memory for these devices to at least the specified minimum, or adjusting the model config.



🔄 Đang load mô hình Qwen...


Loading checkpoint shards:   0%|          | 0/37 [00:00<?, ?it/s]

In [53]:
def standard_prompt(query, task_type="elementary_math"):
    """
    Create a standardized prompt for elementary school math problems.
    
    Parameters:
    - query (str): The math problem statement
    - task_type (str): Type of task (default: "elementary_math")
    
    Returns:
    - str: Formatted prompt ready to be sent to an LLM
    """
    if task_type == "elementary_math":
        return f"""Please solve this elementary school math problem step by step:

Problem: {query}

Please follow these steps:
1. Read the problem carefully
2. Identify what is given and what needs to be found
3. Write down the equation or method needed
4. Solve step by step
5. Check your answer
6. Write the final answer clearly

Solution:"""
    else:
        return f"""Please solve this problem:

Problem: {query}

Answers:"""

In [54]:
@torch.no_grad()
def clear_gpu_cache():
    """Clear GPU cache and garbage collection"""
    torch.cuda.empty_cache()
    gc.collect()
    print("🧹 Đã giải phóng bộ nhớ cache")

class InferenceContext:
    """Context manager for automatic memory management"""
    def __enter__(self):
        clear_gpu_cache()
        return self
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        clear_gpu_cache()

def optimize_inference(model, tokenizer, problem_text, max_new_tokens=50):
    """
    Optimized inference function with performance improvements
    """
    try:
        # 1. Optimize input processing
        inputs = tokenizer(
            problem_text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(model.device)

        # 2. Enable optimizations
        with torch.inference_mode(), torch.amp.autocast('cuda'):
            outputs = model.generate(
                inputs.input_ids,
                attention_mask=inputs.attention_mask,
                # 3. Optimize generation parameters
                max_new_tokens=max_new_tokens,
                min_new_tokens=10,
                temperature=0.7,
                top_p=0.9,
                top_k=50,
                num_beams=1,
                do_sample=False,
                early_stopping=True,
                pad_token_id=model.config.eos_token_id,
                # 4. Enable optimization flags
                use_cache=True,
                repetition_penalty=1.0
            )
        
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return result

    except Exception as e:
        return f"Lỗi: {str(e)}"

def batch_inference(model, tokenizer, questions, batch_size=4):
    """
    Process multiple questions in batches for better performance
    """
    results = []
    for i in range(0, len(questions), batch_size):
        batch = questions[i:i + batch_size]
        inputs = tokenizer(
            batch, 
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(model.device)
        
        with torch.inference_mode():
            outputs = model.generate(**inputs)
            
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        results.extend(decoded)
    return results

def fast_inference(model, tokenizer, question):
    """
    Fast inference wrapper with automatic memory management
    """
    with InferenceContext():
        # Clear cache before starting
        clear_gpu_cache()
        
        # Perform inference with optimizations
        result = optimize_inference(
            model,
            tokenizer,
            question,
            max_new_tokens=50
        )
        
        return result

In [55]:
# Function to call GPT API
def gpt_inference(problem_text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": standard_prompt(problem_text)}],
            max_tokens=50
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        return f"API Error: {str(e)}"

In [56]:
import google.generativeai as genai
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# ✅ **Load mô hình Gemini**
try:
    gemini_text_model = genai.GenerativeModel('gemini-1.5-flash-8b')  # Thử model khác nếu lỗi
    gemini_vision_model = genai.GenerativeModel('gemini-1.5-flash')
    print("✅ Gemini models loaded successfully!")
except Exception as e:
    print(f"❌ Lỗi khi load Gemini: {e}")


✅ Gemini models loaded successfully!


In [57]:
def gemini_inference_improved(problem_text, model_type='text'):
    """Hàm inference cải tiến cho mô hình Gemini"""
    try:
        prompt = standard_prompt(problem_text, "math")
        
        if model_type == 'text':
            response = gemini_text_model.generate_content(
                prompt,
                generation_config={
                    "temperature": 0.7,
                    "max_output_tokens": 512,
                    "top_p": 0.95
                }
            )
        else:
            # Lỗi: problem_image không được định nghĩa
            # Sửa: bỏ tham chiếu tới problem_image
            response = gemini_vision_model.generate_content(
                prompt,  # Chỉ sử dụng văn bản
                generation_config={
                    "temperature": 0.7,
                    "max_output_tokens": 512,
                    "top_p": 0.95
                }
            )
        
        return response.text.replace("**", "").strip()  # Clean formatting
    
    except Exception as e:
        print(f"⚠️ Lỗi Gemini: {str(e)}")
        return "Không thể tạo phản hồi từ Gemini"

In [58]:
def qwen_inference_improved(problem_text):
    """Improved inference function for Qwen model"""
    if qwen_model is None or qwen_tokenizer is None:
        return "Lỗi: Model hoặc tokenizer Qwen chưa được load"
    
    try:
        with InferenceContext():
            result = optimize_inference(
                qwen_model,
                qwen_tokenizer,
                problem_text,
                max_new_tokens=50
            )
            return result
    except Exception as e:
        return f"Lỗi Qwen: {str(e)}"


In [59]:

def llama_inference_improved(problem_text):
    """Improved inference function for Llama model"""
    if llama_model is None or llama_tokenizer is None:
        return "Lỗi: Model hoặc tokenizer Llama chưa được load"
    
    try:
        with InferenceContext():
            result = optimize_inference(
                llama_model,
                llama_tokenizer,
                problem_text,
                max_new_tokens=50
            )
            return result
    except Exception as e:
        return f"Lỗi Llama: {str(e)}"

In [60]:

print("\n" + "="*60)
print("🚀 BƯỚC 2: THỬ INFERENCE VỚI MÔ HÌNH ĐÃ CẢI THIỆN")
print("="*60)


#%% [Example Usage]
problem1 = "Đề bài: Một con trâu ăn cỏ trong 5 ngày thì hết một đám cỏ. Nếu hai con trâu ăn thì chỉ đủ trong 3 ngày. Hỏi nếu ba con trâu ăn thì đám cỏ đó đủ trong mấy ngày?"
problem2 = "Đề bài: Có một ông lão muốn chia bó đũa cho các con sao cho: • Nếu chia cho 2 người thì dư 1 chiếc. • Nếu chia cho 3 người thì dư 1 chiếc. • Nếu chia cho 4, 5 hoặc 6 người thì đều dư 1 chiếc. • Nhưng nếu chia cho 7 người thì vừa đủ. Hỏi bó đũa có bao nhiêu chiếc?"
problem3 = "Đề bài: Một người gánh cam đi bán, khi qua chợ, ông bán một nửa số cam và thêm nửa quả. Đi tiếp một đoạn, ông lại bán một nửa số cam còn lại và thêm nửa quả. Cuối cùng, khi đến chợ cuối cùng, ông lại bán một nửa số cam còn lại và thêm nửa quả, hết sạch cam. Hỏi ban đầu ông có bao nhiêu quả cam?"
problem4 = "Đề bài: Vừa gà vừa chó, Bó lại cho tròn, Ba mươi sáu con, Một trăm chân chẵn. Hỏi có bao nhiêu con gà, bao nhiêu con chó?"

# Chạy inference với mô hình Gemini (thường ổn định nhất)
print("\n🤖 Đang chạy inference với Gemini...")
gemini_result = gemini_inference_improved(problem4)
print("\n📝 Kết quả Gemini:")
print(gemini_result)

# Thử với Llama
print("\n🤖 Đang chạy inference với Llama cải tiến...")
# Giải phóng bộ nhớ trước khi chạy
clear_memory()
try:
    llama_result = llama_inference_improved(problem4)
    print("\n📝 Kết quả Llama:")
    print(llama_result)
except Exception as e:
    print(f"❌ Không thể chạy inference với Llama: {e}")

# Thử với Qwen
print("\n🤖 Đang chạy inference với Qwen cải tiến...")
# Giải phóng bộ nhớ trước khi chạy
clear_memory()
try:
    qwen_result = qwen_inference_improved(problem4)
    print("\n📝 Kết quả Qwen:")
    print(qwen_result)
except Exception as e:
    print(f"❌ Không thể chạy inference với Qwen: {e}")

# Tổng kết
# Thêm kiểm tra biến
llama_result = "Không có kết quả" if 'llama_result' not in locals() else llama_result
qwen_result = "Không có kết quả" if 'qwen_result' not in locals() else qwen_result
gemini_result = "Không có kết quả" if 'gemini_result' not in locals() else gemini_result

# Tổng kết
print("\n" + "="*60)
print("✅ HOÀN THÀNH")
print("="*60)
print("📊 Tóm tắt kết quả:")
print(f"- Gemini: {'Thành công' if 'Không thể tạo phản hồi từ Gemini' not in gemini_result else 'Thất bại'}")
print(f"- Llama: {'Thành công' if 'Lỗi Llama' not in llama_result else 'Thất bại'}")
print(f"- Qwen: {'Thành công' if 'Lỗi Qwen' not in qwen_result else 'Thất bại'}")


🚀 BƯỚC 2: THỬ INFERENCE VỚI MÔ HÌNH ĐÃ CẢI THIỆN

🤖 Đang chạy inference với Gemini...

📝 Kết quả Gemini:
Let 'g' be the number of chickens and 'h' be the number of dogs.

We have two equations:

1) g + h = 36  (Total animals)
2) 2g + 4h = 100 (Total legs)

From equation 1, we can express g as:

g = 36 - h

Substitute this into equation 2:

2(36 - h) + 4h = 100
72 - 2h + 4h = 100
2h = 28
h = 14

Now substitute the value of h back into the equation for g:

g = 36 - 14
g = 22

Therefore, there are $\boxed{22}$ chickens and $\boxed{14}$ dogs.

🤖 Đang chạy inference với Llama cải tiến...
🧹 Đã giải phóng bộ nhớ cache

📝 Kết quả Llama:
Lỗi: Model hoặc tokenizer Llama chưa được load

🤖 Đang chạy inference với Qwen cải tiến...
🧹 Đã giải phóng bộ nhớ cache
🧹 Đã giải phóng bộ nhớ cache


g:\Trune\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
g:\Trune\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
g:\Trune\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:677: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


🧹 Đã giải phóng bộ nhớ cache

📝 Kết quả Qwen:
Lỗi: CUDA out of memory. Tried to allocate 2.32 GiB. GPU 0 has a total capacity of 47.99 GiB of which 0 bytes is free. Of the allocated memory 45.53 GiB is allocated by PyTorch, and 13.69 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

✅ HOÀN THÀNH
📊 Tóm tắt kết quả:
- Gemini: Thành công
- Llama: Thành công
- Qwen: Thành công


In [61]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def safe_inference(model, tokenizer, text, model_name):
    """
    Safe inference wrapper with error handling and logging
    """
    try:
        if model is None or tokenizer is None:
            logger.error(f"{model_name}: Model or tokenizer not loaded")
            return f"Lỗi {model_name}: Model chưa được load"
            
        with InferenceContext():
            result = optimize_inference(
                model,
                tokenizer,
                text,
                max_new_tokens=50
            )
            logger.info(f"{model_name}: Inference successful")
            return result
            
    except Exception as e:
        logger.error(f"{model_name}: Error during inference - {str(e)}")
        return f"Lỗi {model_name}: {str(e)}"

In [62]:
def check_gpu_memory():
    """Check GPU memory before inference"""
    for i in range(torch.cuda.device_count()):
        total = torch.cuda.get_device_properties(i).total_memory / 1024**3
        allocated = torch.cuda.memory_allocated(i) / 1024**3
        free = total - allocated
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Total: {total:.2f} GB")
        print(f"  Allocated: {allocated:.2f} GB")
        print(f"  Free: {free:.2f} GB")

In [63]:
def cot_prompt(problem):
    return f"""You are a helpful assistant tasked with solving the following problem step by step. Explain your reasoning clearly at each stage and provide the final answer.

Problem: {problem}

Follow these steps to solve the problem:
1. **Restate the Problem**: Rephrase the problem in your own words to confirm understanding.
2. **Identify Key Variables**: Define the unknowns (e.g., 'let x be...') and list what needs to be solved.
3. **List Given Information**: Write down all provided data and any assumptions (e.g., units, implied conditions).
4. **Recall Relevant Concepts**: Mention any formulas, rules, or strategies (e.g., algebra, geometry) that apply.
5. **Break Down the Problem**: If complex, split it into smaller sub-problems or logical steps.
6. **Solve Step by Step**: Work through each part, showing all calculations and explaining your logic.
7. **Combine Results**: Integrate the solutions to sub-problems to find the overall answer.
8. **Verify the Solution**: Check the answer by substituting it back into the problem or using an alternative method. Explain why it makes sense.
9. **Present the Answer**: State the final answer clearly and concisely.

Focus on clarity and logical progression. If unsure, explore multiple approaches and select the best one."""